## Data

This dataset was originally posted on Kaggle. The key task is to predict whether a product/part will go on backorder.

Product backorder may be the result of strong sales performance (e.g. the product is in such a high demand that production cannot keep up with sales). However, backorders can upset consumers, lead to canceled orders and decreased customer loyalty. Companies want to avoid backorders, but also avoid overstocking every product (leading to higher inventory costs).

This dataset has ~1.9 million observations of products/parts in an 8 week period. The source of the data is unreferenced.

* __Outcome__: whether the product went on backorder
* __Predictors__: Current inventory, sales history, forecasted sales, recommended stocking amount, product risk flags etc. (22 predictors in total)

The features with the target variable of the dataset are as follows:

**Description**
~~~
sku - Random ID for the product
national_inv - Current inventory level for the part
lead_time - Transit time for product (if available)
in_transit_qty - Amount of product in transit from source
forecast_3_month - Forecast sales for the next 3 months
forecast_6_month - Forecast sales for the next 6 months
forecast_9_month - Forecast sales for the next 9 months
sales_1_month - Sales quantity for the prior 1 month time period
sales_3_month - Sales quantity for the prior 3 month time period
sales_6_month - Sales quantity for the prior 6 month time period
sales_9_month - Sales quantity for the prior 9 month time period
min_bank - Minimum recommend amount to stock
potential_issue - Source issue for part identified
pieces_past_due - Parts overdue from source
perf_6_month_avg - Source performance for prior 6 month period
perf_12_month_avg - Source performance for prior 12 month period
local_bo_qty - Amount of stock orders overdue
deck_risk - Part risk flag
oe_constraint - Part risk flag
ppap_risk - Part risk flag
stop_auto_buy - Part risk flag
rev_stop - Part risk flag

went_on_backorder - Product actually went on backorder. (Target variable)
~~~

Two data files are given and the files are accessible in the JupyterHub environment:
 * `/dsa/data/all_datasets/back_order/Kaggle_Training_Dataset_v2.csv`
 * `/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`


 
<span style='background:yellow'>**NOTE:** The training data file is 117MB. **Do NOT try to version control any data files** (training, test, or created), you may blow-through the _push limit_.</span>  
You can easily lock up a notebook with bad coding practices.  
Please save you project early, and often, and use `git commits` to checkpoint your process.

## Exploration, Training, and Validation

You will examine the _training_ dataset and perform 
 * **data preparation and exploratory data analysis**, 
 * **anomaly detection / removal**,
 * **dimensionality reduction** and then
 * **train and validate**.

We aim to develop at least 3 unique pipelines. By unique we mean that if a classification (or feature selection/anomaly detection) method is used in Pipeline 1, that classification method should not be used in Pipeline 2 and Pipeline 3. 

Of the 3 different models, you are free to pick any models from Scikit-Learn or any custom models that work within sklearn pipeline.

### Validation Assessment

Your first, intermediate, result will be an **assessment** of the models' performance.
This assessement should be grounded within a 5-fold or 10-fold cross-validation methodology. Give an unbiased evaluaiton of the best model within each pipeline. This should include the confusion matrix, precision, recall, F1-score, and accuracy for each classifier as a bare minimum.

## Testing

Once we have chosen our final model, we need to re-train it using all the training data. Then final evaluation should be performed on the given test dataset. 





--- 
##  Overview / Roadmap

**General steps**:
* Part I: Preprocessing
  * Dataset carpentry & Exploratory Data Analysis
    * Develop functions to perform the necessary steps, you will have to carpentry the Training and the Testing data.
  * Generate a **smart sample** of the the data
* Part II: Training and Validation
  * Create 3 alternative pipelines, each does:
      * Anomaly detection
      * Dimensionality reduction
      * Classification
* Part III: Testing
  * Train chosen model full training data
  * Evaluate model against testing
  * Write a summary of your processing and an analysis of the model performance




# Part I: Data Preprocessing

In this part, we preprocess the given training set. 


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd

## Load dataset

**Description**
~~~
sku - Random ID for the product
national_inv - Current inventory level for the part
lead_time - Transit time for product (if available)
in_transit_qty - Amount of product in transit from source
forecast_3_month - Forecast sales for the next 3 months
forecast_6_month - Forecast sales for the next 6 months
forecast_9_month - Forecast sales for the next 9 months
sales_1_month - Sales quantity for the prior 1 month time period
sales_3_month - Sales quantity for the prior 3 month time period
sales_6_month - Sales quantity for the prior 6 month time period
sales_9_month - Sales quantity for the prior 9 month time period
min_bank - Minimum recommend amount to stock
potential_issue - Source issue for part identified
pieces_past_due - Parts overdue from source
perf_6_month_avg - Source performance for prior 6 month period
perf_12_month_avg - Source performance for prior 12 month period
local_bo_qty - Amount of stock orders overdue
deck_risk - Part risk flag
oe_constraint - Part risk flag
ppap_risk - Part risk flag
stop_auto_buy - Part risk flag
rev_stop - Part risk flag
went_on_backorder - Product actually went on backorder. 
~~~

**Note**: This is a real-world dataset without any preprocessing.  
There will also be warnings due to fact that the 1st column is mixing integer and string values.  
**NOTE:** The last column, `went_on_backorder`, is what we are trying to predict.


In [2]:
# Dataset location
DATASET = '/dsa/data/all_datasets/back_order/Kaggle_Training_Dataset_v2.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)

dataset.head().transpose()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
sku,1523511,1806635,1371892,2113996,1895659
national_inv,7.0,10.0,0.0,0.0,182.0
lead_time,12.0,2.0,2.0,2.0,2.0
in_transit_qty,0.0,0.0,0.0,0.0,13.0
forecast_3_month,0.0,0.0,15.0,60.0,299.0
forecast_6_month,0.0,0.0,15.0,60.0,571.0
forecast_9_month,0.0,0.0,15.0,80.0,916.0
sales_1_month,0.0,0.0,0.0,0.0,57.0
sales_3_month,0.0,0.0,0.0,0.0,209.0
sales_6_month,0.0,0.0,0.0,0.0,429.0


In [3]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
national_inv,1687860.0,496.111782,29615.233831,-27256.0,4.00,15.00,80.00,12334404.0
lead_time,1586967.0,7.872267,7.056024,0.0,4.00,8.00,9.00,52.0
in_transit_qty,1687860.0,44.052022,1342.741731,0.0,0.00,0.00,0.00,489408.0
forecast_3_month,1687860.0,178.119284,5026.553102,0.0,0.00,0.00,4.00,1427612.0
forecast_6_month,1687860.0,344.986664,9795.151861,0.0,0.00,0.00,12.00,2461360.0
forecast_9_month,1687860.0,506.364431,14378.923562,0.0,0.00,0.00,20.00,3777304.0
sales_1_month,1687860.0,55.926069,1928.195879,0.0,0.00,0.00,4.00,741774.0
sales_3_month,1687860.0,175.025930,5192.377625,0.0,0.00,1.00,15.00,1105478.0
sales_6_month,1687860.0,341.728839,9613.167104,0.0,0.00,2.00,31.00,2146625.0
sales_9_month,1687860.0,525.269701,14838.613523,0.0,0.00,4.00,47.00,3205172.0


## Processing

In this section, the goal is to figure out:

* which columns we can use directly,  
* which columns are usable after some processing,  
* and which columns are not processable or obviously irrelevant (like product id) that we will discard.

Then process and prepare this dataset for creating a predictive model.

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Data columns (total 23 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   sku                1687861 non-null  object 
 1   national_inv       1687860 non-null  float64
 2   lead_time          1586967 non-null  float64
 3   in_transit_qty     1687860 non-null  float64
 4   forecast_3_month   1687860 non-null  float64
 5   forecast_6_month   1687860 non-null  float64
 6   forecast_9_month   1687860 non-null  float64
 7   sales_1_month      1687860 non-null  float64
 8   sales_3_month      1687860 non-null  float64
 9   sales_6_month      1687860 non-null  float64
 10  sales_9_month      1687860 non-null  float64
 11  min_bank           1687860 non-null  float64
 12  potential_issue    1687860 non-null  object 
 13  pieces_past_due    1687860 non-null  float64
 14  perf_6_month_avg   1687860 non-null  float64
 15  perf_12_month_avg  1687860 non-n

### Take samples and examine the dataset

In [5]:
dataset.iloc[:3,:6]

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month
0,1523511,7.0,12.0,0.0,0.0,0.0
1,1806635,10.0,2.0,0.0,0.0,0.0
2,1371892,0.0,2.0,0.0,15.0,15.0


In [6]:
dataset.iloc[:3,6:12]

,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,15.0,0.0,0.0,0.0,0.0,1.0


In [7]:
dataset.iloc[:3,12:18]

,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk
0,No,0.0,0.63,0.72,0.0,No
1,No,0.0,0.46,0.61,0.0,Yes
2,No,5.0,1.00,1.00,0.0,Yes


In [8]:
dataset.iloc[:3,18:24]

,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,No,No,Yes,No,No
1,No,No,Yes,No,No
2,No,No,Yes,No,No


### Drop columns that are obviously irrelevant or not processable

In [9]:
# Add code below this comment  (Question #E101)
# ----------------------------------

dataset = dataset.drop(columns=['sku', 'lead_time']) # I dropped sku because it is irrelevant, and lead_time, because it
                                                     # contains many Nan values and is numeric.
                                                     # the yes/no columns can be made processable by converting into binary.



### Find unique values of string columns

Now try to make sure that these Yes/No columns really only contains Yes or No.  
If that's true, proceed to convert them into binaries (0s and 1s).

**Tip**: use [unique()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html) function of pandas Series.

Example

~~~python
print('went_on_backorder', dataset['went_on_backorder'].unique())
~~~

In [10]:
# All the column names of these yes/no columns
yes_no_columns = list(filter(lambda i: dataset[i].dtype!=np.float64, dataset.columns))
print(yes_no_columns)

# Add code below this comment  (Question #E102)
# ----------------------------------

dataset.potential_issue.unique()

['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']


array(['No', 'Yes', nan], dtype=object)

In [11]:
dataset.deck_risk.unique()

array(['No', 'Yes', nan], dtype=object)

In [12]:
dataset.oe_constraint.unique()

array(['No', 'Yes', nan], dtype=object)

In [13]:
dataset.ppap_risk.unique()

array(['No', 'Yes', nan], dtype=object)

In [14]:
dataset.stop_auto_buy.unique()

array(['Yes', 'No', nan], dtype=object)

In [15]:
dataset.rev_stop.unique()

array(['No', 'Yes', nan], dtype=object)

In [16]:
dataset.went_on_backorder.unique()

array(['No', 'Yes', nan], dtype=object)

You may see **nan** also as possible values representing missing values in the dataset.

We fill them using most popular values, the [Mode](https://en.wikipedia.org/wiki/Mode_%28statistics%29) in Stats.

In [17]:
for column_name in yes_no_columns:
    mode = dataset[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    dataset[column_name].fillna(mode, inplace=True)

Filling missing values of potential_issue with No
Filling missing values of deck_risk with No
Filling missing values of oe_constraint with No
Filling missing values of ppap_risk with No
Filling missing values of stop_auto_buy with Yes
Filling missing values of rev_stop with No
Filling missing values of went_on_backorder with No


### Convert yes/no columns into binary (0s and 1s)

In [18]:
# Add code below this comment  (Question #E103)
# ----------------------------------

dataset.potential_issue = dataset.potential_issue.map(dict(Yes=1, No=0))
dataset.deck_risk = dataset.deck_risk.map(dict(Yes=1, No=0))
dataset.oe_constraint = dataset.oe_constraint.map(dict(Yes=1, No=0))
dataset.ppap_risk = dataset.ppap_risk.map(dict(Yes=1, No=0))
dataset.stop_auto_buy = dataset.stop_auto_buy.map(dict(Yes=1, No=0))
dataset.rev_stop = dataset.rev_stop.map(dict(Yes=1, No=0))
dataset.went_on_backorder = dataset.went_on_backorder.map(dict(Yes=1, No=0))


Now all columns should be either int64 or float64.

In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Data columns (total 21 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   national_inv       1687860 non-null  float64
 1   in_transit_qty     1687860 non-null  float64
 2   forecast_3_month   1687860 non-null  float64
 3   forecast_6_month   1687860 non-null  float64
 4   forecast_9_month   1687860 non-null  float64
 5   sales_1_month      1687860 non-null  float64
 6   sales_3_month      1687860 non-null  float64
 7   sales_6_month      1687860 non-null  float64
 8   sales_9_month      1687860 non-null  float64
 9   min_bank           1687860 non-null  float64
 10  potential_issue    1687861 non-null  int64  
 11  pieces_past_due    1687860 non-null  float64
 12  perf_6_month_avg   1687860 non-null  float64
 13  perf_12_month_avg  1687860 non-null  float64
 14  local_bo_qty       1687860 non-null  float64
 15  deck_risk          1687861 non-n

In [20]:
dataset.head

<bound method NDFrame.head of          national_inv  in_transit_qty  forecast_3_month  forecast_6_month  \
0                 7.0             0.0               0.0               0.0   
1                10.0             0.0               0.0               0.0   
2                 0.0             0.0              15.0              15.0   
3                 0.0             0.0              60.0              60.0   
4               182.0            13.0             299.0             571.0   
...               ...             ...               ...               ...   
1687856          11.0             0.0               0.0               0.0   
1687857           2.0             0.0               0.0               0.0   
1687858          60.0             0.0               0.0               0.0   
1687859         304.0             0.0               0.0               0.0   
1687860          12.0             0.0               0.0               0.0   

         forecast_9_month  sales_1_month  sal

### Smartly sample the data into a more manageable size for cross-fold validation in Grid Search

**Note**: This is a good point to re-balance dataset before actually moving on. For sampling we can either take advantage of pandas/numpy `sample` method or use `imblearn` [package](https://imbalanced-learn.org/stable/user_guide.html#user-guide). 

`imblearn` module has implemented a pipeline on top sklearn pipeline, and it is possible to add sampling strategies within the `imblearn` pipeline. We are not required to use `imblearn` pipeline for this project. 

In [21]:
num_backorder = np.sum(dataset['went_on_backorder']==1)
print('backorder ratio:', num_backorder, '/', len(dataset), '=', num_backorder / len(dataset))

backorder ratio: 11293 / 1687861 = 0.006690716830355106


Create a smar sample of the dataset. You can either store the data to csv files or simply use `joblib` to dump the variables and load them in Part 2. 

**Example code for using joblib:**

Say we need to store three objects (sampled_X, sampled_y, model) to a file. 

```python
import joblib

# for dumping 
joblib.dump([sampled_X, sampled_y, model], 'data/sample-data-v1.pkl')

# for loading
sampled_X, sampled_y, model = joblib.load('data/sample-data-v1.pkl')

```


In [22]:
no_backorder = dataset[dataset['went_on_backorder']==0]
yes_backorder = dataset[dataset['went_on_backorder']==1]

In [24]:
from sklearn.utils import resample

backorder_upsampled = resample(yes_backorder,
                          replace=True, # sample with replacement
                          n_samples=no_backorder.shape[0], # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
ds_upsampled = pd.concat([no_backorder, backorder_upsampled])

# check new class counts
ds_upsampled.went_on_backorder.value_counts()

# Upsampling the minority class doesn't make much sense, so instead I'll use downsampling for the majority

0    1676568
1    1676568
Name: went_on_backorder, dtype: int64

In [25]:
# downsample majority
no_backorder_downsampled = resample(no_backorder,
                                replace = False, # sample without replacement
                                n_samples = yes_backorder.shape[0], # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
ds_downsampled = pd.concat([no_backorder_downsampled, yes_backorder])

# checking counts
ds_downsampled.went_on_backorder.value_counts()

0    11293
1    11293
Name: went_on_backorder, dtype: int64

In [26]:
# Add code below this comment   (Question #E104) 
# ----------------------------------

X_smart = np.array(ds_downsampled.iloc[:,:-1])
y_smart = np.array(ds_downsampled.went_on_backorder)


**Note:** After sampling the data, you may want to write the data to a file for reloading later.

<span style="background: yellow;">If required, remove the `dataset` variable to avoid any memory-related issue.</span> 

In [27]:
# Write your smart sampling to local file  
# ----------------------------------
import joblib

joblib.dump([ds_downsampled, X_smart, y_smart], 'sample-data.pkl')



['sample-data.pkl']

You should have made a couple commits so far of this project.  
**Definitely make a commit of the notebook now!**  
Comment should be: `Final Project, Checkpoint - Data Sampled`


# Save your notebook!
## Then `File > Close and Halt`